# Single Output Regression

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

SEED = 42
HOME = Path.cwd().parent
RAW_PATH = HOME / 'data/raw'
print(RAW_PATH)
MAX_TIME = 500

/home/rco/DS/kdd22/data/raw


In [2]:
pub = pd.read_csv(RAW_PATH / 'public.csv')
print(pub.shape)
pub.head()

(146262, 6)


,Filename,Altitude,Delta,North,East,kfold
0,00003e3b9e5336685200ae85d21b4f5e.jpg,178.829834,-0.065231,-0.386045,0.929772,0
1,0001261e2060303a06ba6c64d676d639.jpg,207.921478,-0.080688,0.635584,0.152819,2
2,0002ac0d783338cfeab0b2bdbd872cda.jpg,178.048431,0.021576,-1.228229,-0.499388,3
3,0004289ee1c7b8b08c77e19878106ae3.jpg,201.084625,0.505981,-1.739709,-0.699928,1
4,0004d0b59e19461ff126e3a08a814c33.jpg,187.550201,-0.328156,-0.169798,2.828752,0


In [3]:
def merge_pixels_dist(dataset='train', suffixes=['', '_a', '_a_est', '_h', '_h_est', '_p', '_p_est', '_rco']):
    df_final = pub.copy()
    for s in suffixes:
        filename = 'pixels_dist_' + dataset + s + '.csv'
        df_right = pd.read_csv(RAW_PATH / filename)
        df_right.rename({'Images':'Filename'}, axis=1, inplace=True)
        df_final = df_final.merge(df_right, on='Filename', how='inner', suffixes=(None, s))
        print(df_final.shape)

    return df_final

train = merge_pixels_dist('train')
train.head()

(91231, 8)
(91231, 10)
(91231, 12)
(91231, 14)
(91231, 16)
(91231, 18)
(91231, 20)
(91231, 40)


,Filename,Altitude,Delta,North,East,kfold,North_pixel,East_pixel,North_pixel_a,East_pixel_a,...,pxl_10,pxl_11,pxl_12,pxl_13,pxl_14,pxl_15,pxl_16,pxl_17,pxl_18,pxl_19
0,00003e3b9e5336685200ae85d21b4f5e.jpg,178.829834,-0.065231,-0.386045,0.929772,0,-1.098183,2.828369,-1.296189,2.868935,...,-2.882545,-1.254211,-3.426113,-1.640629,-2.829666,-1.183493,-2.998511,-1.221453,-2.975761,-1.224934
1,0001261e2060303a06ba6c64d676d639.jpg,207.921478,-0.080688,0.635584,0.152819,2,1.452448,0.207358,0.847966,0.356077,...,-0.268454,0.959177,-0.203712,1.094475,-0.268454,0.959177,-0.252202,1.062943,-0.326046,0.971859
2,0002ac0d783338cfeab0b2bdbd872cda.jpg,178.048431,0.021576,-1.228229,-0.499388,3,-3.544215,-1.494362,-3.517452,-1.592591,...,1.601097,-3.896843,1.576309,-3.810501,1.554550,-3.741711,1.583000,-3.806505,1.571258,-3.768505
3,0004289ee1c7b8b08c77e19878106ae3.jpg,201.084625,0.505981,-1.739709,-0.699928,1,-6.054061,-4.082150,-6.091535,-3.099275,...,3.585079,-6.579813,1.656113,-5.524193,2.281319,-5.760706,1.842140,-5.587303,2.933319,-6.275887
4,0004d0b59e19461ff126e3a08a814c33.jpg,187.550201,-0.328156,-0.169798,2.828752,0,-0.690250,8.232869,-0.449659,8.125691,...,-7.957325,-0.792717,-7.955856,-0.466026,-7.913773,-0.495544,-7.805359,-0.389091,-7.685028,-0.443871


In [4]:
test = merge_pixels_dist('test')
test.drop(['North', 'East'], axis=1, inplace=True)
test.head()

(55031, 8)
(55031, 10)
(55031, 12)
(55031, 14)
(55031, 16)
(55031, 18)
(55031, 20)
(55031, 40)


,Filename,Altitude,Delta,kfold,North_pixel,East_pixel,North_pixel_a,East_pixel_a,North_pixel_a_est,East_pixel_a_est,...,pxl_10,pxl_11,pxl_12,pxl_13,pxl_14,pxl_15,pxl_16,pxl_17,pxl_18,pxl_19
0,000053b1e684c9e7ea73727b2238ce18.jpg,167.943069,0.010269,0,3.946422,-3.815853,0.356597,0.257315,-0.654044,-1.042109,...,1.148582,-3.340916,0.824883,-3.930728,-8.145741,16.754318,-0.074838,0.269563,-0.299238,-1.830235
1,00029153d12ae1c9abe59c17ff2e0895.jpg,195.853088,0.089218,2,7.668228,-4.031865,9.366933,-2.493681,4.654472,-5.100857,...,4.385246,3.600212,3.636406,2.604145,4.080269,3.412636,4.182144,2.752205,2.429424,1.857857
2,0006246bee639c7a7b11a08e34dd3cc6.jpg,146.943466,-0.018326,1,-11.549439,-0.880645,-12.518370,-0.683746,-12.124315,-0.030847,...,0.003180,-15.928162,0.132889,-11.713745,0.002014,-11.649620,-0.202403,-11.936136,-0.197823,-11.768921
3,00063cb5da1826febf178b669eea3250.jpg,213.184418,-0.108704,1,-28.991636,1.930891,-25.942930,-0.682081,-21.449981,4.212570,...,9.191109,-32.700813,-16.807194,-2.444351,-15.071526,2.416723,-1.628705,-5.106833,-4.408707,-24.759686
4,00063ece2e68a8847f228e8fd922f851.jpg,184.757767,0.017700,0,0.184479,-6.925782,-0.165511,-6.852798,0.008325,-6.209880,...,5.979332,-0.635529,5.925034,-0.113266,6.414247,-0.772942,5.471008,-0.318184,6.608952,-0.145115


In [5]:
import h2o
from h2o.automl import H2OAutoML

h2o.init(nthreads=16, max_mem_size='32G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_342"; OpenJDK Runtime Environment (build 1.8.0_342-8u342-b07-0ubuntu1~22.04-b07); OpenJDK 64-Bit Server VM (build 25.342-b07, mixed mode)
  Starting server from /home/rco/anaconda3/envs/kdd22/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphz4jts7x
  JVM stdout: /tmp/tmphz4jts7x/h2o_rco_started_from_python.out
  JVM stderr: /tmp/tmphz4jts7x/h2o_rco_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,00 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_rco_ulwmom
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,28.42 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [6]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
features = [f for f in train.columns if f not in ['Filename', 'North', 'East', 'kfold']]

def train_automl(train, test, target, max_runtime_secs=3600):
    
    aml = H2OAutoML(seed=SEED,
                    max_runtime_secs=max_runtime_secs,
                    stopping_metric='RMSE',
                    sort_metric='RMSE',
                    exploitation_ratio=0.1)

    aml.train(x=features, y=target, training_frame=train)  
    
    train_preds = aml.leader.predict(train)
    train_preds.rename({'predict':target+'_prediction'})
    
    test_preds = aml.leader.predict(test)
    test_preds.rename({'predict':target+'_prediction'})
    
    return aml, train_preds, test_preds
    

In [8]:
north_aml, north_train_preds, north_test_preds = train_automl(train, test, 'North', MAX_TIME)
north_aml.leaderboard.head(20)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_1_20221031_233610,0.342429,0.117258,0.134669,nan,0.117258
StackedEnsemble_BestOfFamily_4_AutoML_1_20221031_233610,0.347487,0.120747,0.139083,nan,0.120747
StackedEnsemble_AllModels_2_AutoML_1_20221031_233610,0.355987,0.126727,0.145808,nan,0.126727
StackedEnsemble_BestOfFamily_3_AutoML_1_20221031_233610,0.356216,0.12689,0.144992,nan,0.12689
StackedEnsemble_AllModels_1_AutoML_1_20221031_233610,0.357823,0.128037,0.147385,nan,0.128037
StackedEnsemble_BestOfFamily_2_AutoML_1_20221031_233610,0.358653,0.128632,0.148552,nan,0.128632
XRT_1_AutoML_1_20221031_233610,0.3609,0.130249,0.149824,nan,0.130249
DRF_1_AutoML_1_20221031_233610,0.361509,0.130688,0.14773,nan,0.130688
GBM_grid_1_AutoML_1_20221031_233610_model_3,0.361607,0.13076,0.148933,nan,0.13076
XGBoost_grid_1_AutoML_1_20221031_233610_model_5,0.36429,0.132707,0.156305,nan,0.132707


In [9]:
east_aml, east_train_preds, east_test_preds = train_automl(train, test, 'East', MAX_TIME)
east_aml.leaderboard.head(20)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_2_20221031_234432,0.309659,0.0958888,0.129924,nan,0.0958888
StackedEnsemble_BestOfFamily_4_AutoML_2_20221031_234432,0.313809,0.0984761,0.131849,nan,0.0984761
StackedEnsemble_AllModels_2_AutoML_2_20221031_234432,0.321605,0.10343,0.139862,nan,0.10343
StackedEnsemble_AllModels_1_AutoML_2_20221031_234432,0.322205,0.103816,0.140866,nan,0.103816
StackedEnsemble_BestOfFamily_3_AutoML_2_20221031_234432,0.324321,0.105184,0.140645,nan,0.105184
StackedEnsemble_BestOfFamily_2_AutoML_2_20221031_234432,0.325309,0.105826,0.14246,nan,0.105826
GBM_grid_1_AutoML_2_20221031_234432_model_3,0.328261,0.107756,0.144401,nan,0.107756
DRF_1_AutoML_2_20221031_234432,0.328744,0.108072,0.142687,nan,0.108072
XGBoost_grid_1_AutoML_2_20221031_234432_model_3,0.330016,0.108911,0.144207,nan,0.108911
XRT_1_AutoML_2_20221031_234432,0.331764,0.110068,0.145848,nan,0.110068


In [10]:
train_final = train.cbind(north_train_preds)
train_final = train_final.cbind(east_train_preds)
train_final.as_data_frame().to_csv('train_final.csv')

train_final.head(5)

Filename,Altitude,Delta,North,East,kfold,North_pixel,East_pixel,North_pixel_a,East_pixel_a,North_pixel_a_est,East_pixel_a_est,North_pixel_h,East_pixel_h,North_pixel_h_est,East_pixel_h_est,North_pixel_p,East_pixel_p,North_pixel_p_est,East_pixel_p_est,pxl_0,pxl_1,pxl_2,pxl_3,pxl_4,pxl_5,pxl_6,pxl_7,pxl_8,pxl_9,pxl_10,pxl_11,pxl_12,pxl_13,pxl_14,pxl_15,pxl_16,pxl_17,pxl_18,pxl_19,North_prediction,East_prediction
00003e3b9e5336685200ae85d21b4f5e.jpg,178.83,-0.0652313,-0.386045,0.929772,0,-1.09818,2.82837,-1.29619,2.86893,-1.22047,2.82652,-1.09818,2.82837,-1.23032,2.83861,-1.29619,2.86893,-1.22047,2.82652,-2.90552,-1.29355,-2.90832,-1.24468,-3.59486,-1.19225,-2.85976,-1.20882,-2.86176,-1.2799,-2.88255,-1.25421,-3.42611,-1.64063,-2.82967,-1.18349,-2.99851,-1.22145,-2.97576,-1.22493,-0.359616,0.916818
0001261e2060303a06ba6c64d676d639.jpg,207.921,-0.0806885,0.635584,0.152819,2,1.45245,0.207358,0.847966,0.356077,0.7925,0.231472,1.45245,0.207358,0.753004,0.217258,0.847966,0.356077,0.7925,0.231472,-0.325096,1.06665,-0.301397,1.05742,-0.30209,1.02171,-0.251114,0.208279,-0.326046,0.971859,-0.268454,0.959177,-0.203712,1.09447,-0.268454,0.959177,-0.252202,1.06294,-0.326046,0.971859,0.616096,0.173137
0002ac0d783338cfeab0b2bdbd872cda.jpg,178.048,0.0215759,-1.22823,-0.499388,3,-3.54421,-1.49436,-3.51745,-1.59259,-3.8237,-1.65885,-3.54421,-1.49436,-3.81837,-1.66816,-3.51745,-1.59259,-3.8237,-1.65885,1.68899,-3.81617,1.60363,-3.97114,1.63071,-3.83078,1.58744,-3.74135,1.53305,-3.82208,1.6011,-3.89684,1.57631,-3.8105,1.55455,-3.74171,1.583,-3.80651,1.57126,-3.76851,-1.17559,-0.479159
0004289ee1c7b8b08c77e19878106ae3.jpg,201.085,0.505981,-1.73971,-0.699928,1,-6.05406,-4.08215,-6.09153,-3.09927,-6.06536,-2.35415,-6.05406,-4.08215,-6.14532,-2.30319,-6.09153,-3.09927,-6.06536,-2.35415,1.72328,-5.50631,-1.37089,-6.69685,2.1552,-5.6545,2.96579,-6.2966,2.36815,-6.12882,3.58508,-6.57981,1.65611,-5.52419,2.28132,-5.76071,1.84214,-5.5873,2.93332,-6.27589,-1.82995,-0.691998
0004d0b59e19461ff126e3a08a814c33.jpg,187.55,-0.328156,-0.169798,2.82875,0,-0.69025,8.23287,-0.449659,8.12569,-0.564889,8.01241,-0.69025,8.23287,-0.555564,7.99966,-0.449659,8.12569,-0.564889,8.01241,-7.93387,-0.467377,-7.94595,-0.667904,-9.20364,-0.8188,-7.78825,-0.459106,-7.8754,-0.574451,-7.95732,-0.792717,-7.95586,-0.466026,-7.91377,-0.495544,-7.80536,-0.389091,-7.68503,-0.443871,-0.153309,2.78272


In [11]:
test_final = test.cbind(north_test_preds)
test_final = test_final.cbind(east_test_preds)
test_final.as_data_frame().to_csv('test_final.csv')

test_final

Filename,Altitude,Delta,kfold,North_pixel,East_pixel,North_pixel_a,East_pixel_a,North_pixel_a_est,East_pixel_a_est,North_pixel_h,East_pixel_h,North_pixel_h_est,East_pixel_h_est,North_pixel_p,East_pixel_p,North_pixel_p_est,East_pixel_p_est,pxl_0,pxl_1,pxl_2,pxl_3,pxl_4,pxl_5,pxl_6,pxl_7,pxl_8,pxl_9,pxl_10,pxl_11,pxl_12,pxl_13,pxl_14,pxl_15,pxl_16,pxl_17,pxl_18,pxl_19,North_prediction,East_prediction
000053b1e684c9e7ea73727b2238ce18.jpg,167.943,0.0102692,0,3.94642,-3.81585,0.356597,0.257315,-0.654044,-1.04211,3.94642,-3.81585,5.85679,-8.51255,0.356597,0.257315,-0.654044,-1.04211,1.14858,-3.34092,-18.8337,17.3706,0.824883,-3.93073,-0.355776,-1.16761,1.00562,-1.44288,1.14858,-3.34092,0.824883,-3.93073,-8.14574,16.7543,-0.0748377,0.269563,-0.299238,-1.83023,-0.163717,-0.187251
00029153d12ae1c9abe59c17ff2e0895.jpg,195.853,0.0892181,2,7.66823,-4.03186,9.36693,-2.49368,4.65447,-5.10086,7.66823,-4.03186,4.9369,-4.14951,9.36693,-2.49368,4.65447,-5.10086,2.75261,2.57346,4.36668,4.15754,4.68184,3.9578,7.69817,2.22164,2.62536,2.22768,4.38525,3.60021,3.63641,2.60415,4.08027,3.41264,4.18214,2.7522,2.42942,1.85786,0.851587,-0.603938
0006246bee639c7a7b11a08e34dd3cc6.jpg,146.943,-0.0183258,1,-11.5494,-0.880645,-12.5184,-0.683746,-12.1243,-0.0308469,-11.5494,-0.880645,-12.1883,0.273252,-12.5184,-0.683746,-12.1243,-0.0308469,-0.169895,-12.0481,-0.13204,-12.0987,-0.0431442,-11.7316,0.027256,-12.1229,-0.0410385,-12.2621,0.00317955,-15.9282,0.132889,-11.7137,0.00201416,-11.6496,-0.202403,-11.9361,-0.197823,-11.7689,-1.90288,0.22419
00063cb5da1826febf178b669eea3250.jpg,213.184,-0.108704,1,-28.9916,1.93089,-25.9429,-0.682081,-21.45,4.21257,-28.9916,1.93089,-21.2372,4.1683,-25.9429,-0.682081,-21.45,4.21257,5.89911,-5.90437,3.89936,-1.86708,-14.7143,-4.84209,-1.69508,-24.1241,-16.3963,24.7725,9.19111,-32.7008,-16.8072,-2.44435,-15.0715,2.41672,-1.62871,-5.10683,-4.40871,-24.7597,-1.33959,-0.170726
00063ece2e68a8847f228e8fd922f851.jpg,184.758,0.0177002,0,0.184479,-6.92578,-0.165511,-6.8528,0.00832452,-6.20988,0.184479,-6.92578,0.00381415,-6.17922,-0.165511,-6.8528,0.00832452,-6.20988,6.3149,-0.0681152,6.29644,-0.425659,6.1424,-0.198502,6.49742,0.0932236,6.58315,0.140034,5.97933,-0.635529,5.92503,-0.113266,6.41425,-0.772942,5.47101,-0.318184,6.60895,-0.145115,0.0816595,-1.52923
000838c1249fec206b77360ff0adc110.jpg,209.136,-0.544525,1,2.54224,-2.43533,2.78873,-2.46239,3.46306,-2.61436,2.54224,-2.43533,3.48015,-2.62869,2.78873,-2.46239,3.46306,-2.61436,2.38117,3.89162,2.36828,3.56954,2.63266,3.54911,2.45233,3.56828,2.45103,3.64938,2.89822,3.66876,2.58473,3.60395,2.91451,3.4528,2.63266,3.54911,2.92265,3.24918,1.50496,-1.51949
000a8e84b013655b832041a6f362e5c9.jpg,177.72,-0.23024,4,-0.601386,-3.88556,-0.061612,-4.1541,-0.321557,-4.38535,-0.601386,-3.88556,-0.280671,-4.39763,-0.061612,-4.1541,-0.321557,-4.38535,3.88873,-0.492369,4.42173,-0.520847,4.33254,-0.263458,4.40611,-0.245636,4.39208,-0.493393,4.26459,-0.423016,4.37817,-0.565887,4.41778,-0.276993,4.09095,-0.462299,4.17667,-0.121765,-0.139893,-1.32363
000d0c74074191add6f22e0004db8f76.jpg,200.488,0.0812378,3,3.12172,-6.69092,2.78086,-5.82139,2.14271,-5.28396,3.12172,-6.69092,2.19983,-5.08691,2.78086,-5.82139,2.14271,-5.28396,6.19389,2.58061,5.38988,2.19228,4.69729,2.00504,4.63818,2.25755,4.80048,1.88295,4.52833,1.88636,4.76211,1.88943,6.22773,2.53867,5.80002,3.7495,6.10212,2.94987,0.755356,-1.85945
000dd3543ac84d906eae52e7c779bb2a.jpg,155.313,0.062027,4,-0.954504,-9.59106,-3.27656,-8.70192,-4.1014,-8.30099,-0.954504,-9.59106,-4.38984,-8.47304,-3.27656,-8.70192,-4.1014,-8.30099,9.49759,-1.18337,7.23822,-5.68969,7.88213,-3.91421,14.9235,-6.43969,8.47189,-4.42818,8.47189,-4.42818,7.81363,-3.73524,8.22931,-4.20756,7.84705,-3.81264,7.23822,-5.68969,-0.48064,-1.06801
00129b07887a18a7331909231c28816e.jpg,187.643,-0.133362,4,0.559739,4.16881,0.2546,4.49667,0.456218,4.28046,0.559739,4.16881,0.446538,4.31604,0.2546,4.49667,0.456218,4.28046,-4.19293,0.394976,-4.23288,0.537773,-4.15697,0.387848,-4.19751,0.569031,-4.252

# Submission

In [12]:
submission = test_final.as_data_frame()[['Filename', 'North_prediction', 'East_prediction']]
submission.rename({'North_prediction':'North', 'East_prediction':'East'}, axis=1, inplace=True)
submission = submission.melt(id_vars='Filename', value_name='Predicted')
submission['Id'] = submission.Filename + ':' + submission.variable
submission = submission[['Id', 'Predicted']]
submission.head()

,Id,Predicted
0,000053b1e684c9e7ea73727b2238ce18.jpg:North,-0.163717
1,00029153d12ae1c9abe59c17ff2e0895.jpg:North,0.851587
2,0006246bee639c7a7b11a08e34dd3cc6.jpg:North,-1.902878
3,00063cb5da1826febf178b669eea3250.jpg:North,-1.339589
4,00063ece2e68a8847f228e8fd922f851.jpg:North,0.081659


In [13]:
submission.to_csv('submission_h2o_rco_full.csv', index=False)

# North Explanation

In [14]:
# north_aml.explain(train)

# East Explanation

In [15]:
# east_aml.explain(train)

In [16]:
h2o.shutdown()

H2O session _sid_9587 closed.


/tmp/ipykernel_226593/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


In [17]:
!kaggle competitions submit -c kddbr-2022 -f submission_h2o_rco_full.csv -m "RCO Lab 5 Odometria Full"

/bin/bash: line 1: kaggle: command not found
